In [1]:
'''
PREPARAÇÃO DOS DADOS
'''

'\nPREPARAÇÃO DOS DADOS\n'

In [2]:
import numpy as np
import pandas as pd
import scipy.optimize  
import os, re, operator
from pprint import pprint
from datetime import datetime

from src.utils import *
from src.k_nearest_neighbors import *
from src.logistic_regression import *
from src.naive_bayes import *
from src.neural_network import *
#from src.support_vector_machines import *
from src.validation import *

#Base de dados
POSITIVE_TRUTHFUL  = ['op_spam_v1.4/positive/truthful/']
POSITIVE_DECEPTIVE = ['op_spam_v1.4/positive/deceptive/']
NEGATIVE_TRUTHFUL  = ['op_spam_v1.4/negative/truthful/']
NEGATIVE_DECEPTIVE = ['op_spam_v1.4/negative/deceptive/']
POSITIVE = [POSITIVE_TRUTHFUL[0], POSITIVE_DECEPTIVE[0]]
NEGATIVE = [NEGATIVE_TRUTHFUL[0], NEGATIVE_DECEPTIVE[0]]
DECEPTIVE = [POSITIVE_DECEPTIVE[0], NEGATIVE_DECEPTIVE[0]]
TRUTHFUL = [POSITIVE_TRUTHFUL[0], NEGATIVE_TRUTHFUL[0]]
ALL = [POSITIVE_TRUTHFUL[0], POSITIVE_DECEPTIVE[0],
           NEGATIVE_TRUTHFUL[0], NEGATIVE_DECEPTIVE[0]]

#Parametros
    #Pre-processamento
USE_NGRAM = True
REMOVE_STOPWORD = True
REMOVE_ABOVE_THRESHOLD = 3
MIN_GRAM = 1
MAX_GRAM = 2
NORMALIZE = False
SAVE = False

    #SVM
COST = [i*0.3+0.1 for i in range(0, 100)]
GAMMA = [i*0.3+0.1 for i in range(0, 100)]
KERNEL = 2

    #FOLD
K_FOLD = 5

    #Outras constants
EMPTY = ''
ZERO = 0
ONE = 1
ENGLISH = 'english'
STOPWORDS = 'stopwords'

if K_FOLD == ZERO:
    raise Exception('Error on parameter for generate folds.')
if MIN_GRAM > MAX_GRAM or MIN_GRAM < ONE or MAX_GRAM < ONE:
    raise Exception('Error on parameter for N-Gram.')
    
print('Preparações realizadas com sucesso.')

Preparações realizadas com sucesso.


In [3]:
# Remoção de Stop Words
import nltk
from nltk.corpus import stopwords

# Download do dicionário de stop wordsgreat
nltk.download(STOPWORDS)

def stopwords_removal(tokens_list):
    '''
        Dado uma lista de tokens.
        Então é removido as stopwords.
        
        Entrada: tokens_list - Uma lista de tokens.
        Saida: new_tokens_list - Uma lista de tokens.
    '''
    if len(tokens_list) == ZERO:
        raise Exception('Error on stopwords_removal.')
        
    new_tokens_list = []
    
    stop_words = stopwords.words(ENGLISH)
    new_tokens_list = [token for token in tokens_list if token not in stop_words]
    
    if len(new_tokens_list) == ZERO:
        raise Exception('Error on stopwords_removal.')
        
    return new_tokens_list

[nltk_data] Downloading package stopwords to C:\Users\Bruno
[nltk_data]     Morii\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# N-gram dos tokens
def generate_ngram(tokens_list):
    '''
        Dado uma lista de tokens.
        Então é gerado N-Gram com base no MAX_GRAM e MIN_GRAM.
        
        Entrada: tokens_list - Uma lista de tokens.
        Saida: allNGrams - Uma lista de N-Gram.
    '''
    if len(tokens_list) == ZERO:
        raise Exception('Error on generate_ngram.')
        
    allNGrams = []
    
    #N-Gram
    for idx in range(MIN_GRAM, MAX_GRAM+1):
        ngrams = zip(*[tokens_list[i:] for i in range(idx)])
        allNGrams += ([" ".join(ngram) for ngram in ngrams])
    
    if len(allNGrams) == ZERO:
        raise Exception('Error on generate_ngram.')
        
    return allNGrams

In [5]:
# Remoção de features irrelevantes por threshold
def features_removal(features_dic, threshold):
    '''
        Dado um dicionario de features e um threshold.
        Então o dicionario é filtrado pelo numero do threshold.
        
        Entrada: features_dic - Um dicionario de features.
                 threshold - Um inteiro para limitar o minimo de ocorrencias de um item do dicionario.
        Saida: new_features_list - Um dicionario de features filtradas.
               new_features_dic - Um dicionario com as features filtradas.
    '''
    if len(features_dic) == ZERO or threshold < ONE:
        raise Exception('Error on features_removal.')

    new_features_dic = {}
    
    #Remove values < threshold
    for key, value in features_dic.items():
        if value > threshold:
            new_features_dic[key] = value

    new_features_list = [k for k, v in new_features_dic.items()]
    new_features_dic = sorted(new_features_dic.items(), key=operator.itemgetter(1))
    
    if len(new_features_list) == ZERO or len(new_features_dic) == ZERO:
        raise Exception('Error on features_removal.')
        
    return new_features_dic, new_features_list

In [6]:
# Tokenização dos textos
def pre_processing(paths, stopwords=False, ngram=False, threshold=0):
    '''
        Dado arquivos de textos.
        Então o texto é limpo (são removidos pontuação) e
            aplicado alguns métodos como remoção de stopwords, N-Gram e filtragem por threshold.
        
        Entrada: paths - Um dicionario de features.
                 stopwords - Um boolean indicando se é para utilizar este método.
                 ngram - Um boolean indicando se é para utilizar este método.
                 threshold - Um inteiro maior que zero que é utilizado na função features_removal.
        Saida: token_list - Uma lista contendo todas as features.
               token_dic - Um dicionario com as features e a sua ocorrencia.
    '''
    if len(paths) == ZERO:
        raise Exception('Error on pre_processing.')
        
    token_dic = {}
    dirs =[]
    token_list = []
    
    #Juntando arquivos de texto
    for p in paths:
        text_path = [p + t for t in os.listdir(p)]
        dirs += (text_path)
    
    for texts in dirs:
        with open(texts, 'r', encoding='utf-8') as stream:
            tokens = []
            text = stream.read()

            #Tokenizando o texto
            tokens = [word for word in (re.sub(r'[^\w\s]+','', text.replace('\n','')).lower().split(' ')) if word != '']
            
            #Tratando stopwords
            if stopwords:
                tokens = stopwords_removal(tokens)
            
            #Aplicando ngram
            if ngram:
                tokens = generate_ngram(tokens)
            
            #Criando dicionario de palavras
            for token in tokens:
                if token in token_dic:
                    token_dic[token] += 1
                else:
                    token_dic[token] = 1
    
    #Removendo palavras com base no threshold
    if threshold > ZERO:
        token_dic, token_list = features_removal(token_dic, threshold)
    
    if len(token_dic) == ZERO or len(token_list) == ZERO:
        raise Exception('Error on pre_processing.')
        
    return token_dic, token_list
print('Pré processamento em andamento...')
start_time = datetime.now()
dicFeatures, listFeatures = pre_processing(DECEPTIVE, REMOVE_STOPWORD, USE_NGRAM, REMOVE_ABOVE_THRESHOLD)
print('Pré-processamento finalizado.')
end_time = datetime.now()
print('Duração: {}'.format(end_time - start_time))
print('Foram gerados {} features.'.format(len(listFeatures)))


Pré processamento em andamento...
Pré-processamento finalizado.
Duração: 0:00:00.695721
Foram gerados 3294 features.


In [7]:
#Gera a matriz de features X e o vetor de classes Y
X = []
Y = []

def generateFeatures(paths, listFeatures, stopwords=False, ngram=False, threshold=0, example_class=0):
    '''
        Dado arquivos de textos.
        Então o dicionario é filtrado pelo numero do threshold
            e é gerado uma matriz que faz a contagem das features por texto.
        
        Entrada: paths - Textos a serem analizados para a geração das features.
                 listFeatures - Uma lista de features.
                 stopwords - Um boolean indicando se é para utilizar este método.
                 ngram - Um boolean indicando se é para utilizar este método.
                 threshold - Um inteiro maior que zero que é utilizado na função features_removal.
        Saida: Nenhuma.
    '''
    if len(paths) == ZERO:
        raise Exception('Error on generateFeatures.')
        
    dirs = []
    
    for p in paths:
        text_path = [p + t for t in os.listdir(p)]
        dirs += (text_path)
    
    for texts in dirs:
        row = np.zeros(len(listFeatures))
        with open(texts, 'r', encoding='utf-8') as stream:
            tokens = []
            text = stream.read()

            #Pre processing
            tokens = [word for word in (re.sub(r'[^\w\s]+','', text.replace('\n','')).lower().split(' ')) if word != '']
            
            if stopwords:
                tokens = stopwords_removal(tokens)
            
            if ngram:
                tokens = generate_ngram(tokens)
            
            for token in tokens:
                for idx, feature in enumerate(listFeatures):
                    if token == feature:
                        row[idx] += 1
                        
            Y.append(example_class)
            X.append(row)
            
start_time = datetime.now()
print('Tratamento em andamento...')

generateFeatures(NEGATIVE_DECEPTIVE, listFeatures, REMOVE_STOPWORD, USE_NGRAM, REMOVE_ABOVE_THRESHOLD, 1)
end_time = datetime.now()
print('Duração - ND: {}'.format(end_time - start_time))

generateFeatures(NEGATIVE_TRUTHFUL, listFeatures, REMOVE_STOPWORD, USE_NGRAM, REMOVE_ABOVE_THRESHOLD, 0)
end_time = datetime.now()
print('Duração - NT: {}'.format(end_time - start_time))

generateFeatures(POSITIVE_DECEPTIVE, listFeatures, REMOVE_STOPWORD, USE_NGRAM, REMOVE_ABOVE_THRESHOLD, 1)
end_time = datetime.now()
print('Duração - PD: {}'.format(end_time - start_time))

generateFeatures(POSITIVE_TRUTHFUL, listFeatures, REMOVE_STOPWORD, USE_NGRAM, REMOVE_ABOVE_THRESHOLD, 0)
end_time = datetime.now()
print('Duração - PT: {}'.format(end_time - start_time))

#Transformando X e Y que são listas em array
X = np.array(X)
Y = np.array(Y)
print('Tratamento finalizado.')

Tratamento em andamento...
Duração - ND: 0:00:11.165826
Duração - NT: 0:00:22.750182
Duração - PD: 0:00:30.586970
Duração - PT: 0:00:38.619988
Tratamento finalizado.


In [8]:
#Salva os dados
if SAVE == True:
    save(np.column_stack((X, Y)), listFeatures, 'matrix_x')
    print('Dados salvos.')

In [9]:
#Normaliza os dados
X_norm = []
mu = 0
sigma = 0

if NORMALIZE == True:
    X_norm, mu, sigma = normalize(X)
    print('Dados normalizados.')

In [10]:
#Gera os folds necessários para os treinamentos 
start_time = datetime.now()
print('Gerando folders...')
folds = stratified_kfolds(Y, K_FOLD, np.unique(Y))
end_time = datetime.now()


print("Foram gerados {} fold(s)".format(K_FOLD), end='\n')
print('Duração: {}\n'.format(end_time - start_time))
idx = 0
for train_index, test_index in folds:
    print('Fold {} - Treino: {}'.format(idx, len(train_index)))
    print('Fold {} - Teste:  {}'.format(idx, len(test_index)), end='\n\n')
    idx += 1
    

Gerando folders...
Foram gerados 5 fold(s)
Duração: 0:00:00.000996

Fold 0 - Treino: 1280
Fold 0 - Teste:  320

Fold 1 - Treino: 1280
Fold 1 - Teste:  320

Fold 2 - Treino: 1280
Fold 2 - Teste:  320

Fold 3 - Treino: 1280
Fold 3 - Teste:  320

Fold 4 - Treino: 1280
Fold 4 - Teste:  320



In [11]:
'''
NAIVE BAYES
'''

'\nNAIVE BAYES\n'

In [12]:
# Multinomial Naive Bayes
def multinomial_naiveBayes(folds, X, Y):
    '''
        Dado os folds, X e Y.
        Então é executado o algoritmo naive bayes para cada fold.
        
        Entrada: folds - Folds para serem testados.
                 X - Matriz de conjunto de dados.
                 Y - Lista com as classes correspondente aos dados da matriz X.
        Saida: resultados - Lista de resultados para cada fold.
               classes - Lista com as classes baseada na lista Y.
    '''
    resultados = []
    classes = np.unique(Y)

    for train_index, test_index in folds:
        # Treinamento
        probsPos, probsNeg = mult_nb_calcular_probabilidades(X[train_index], Y[train_index])

        # Classificação
        pred = []
        for x in X[test_index]:
            pred.append(mult_nb_classificacao(x, probsPos, probsNeg, sum(Y == 1)/len(Y), sum(Y == 0)/len(Y)))

        cm = get_confusionMatrix(Y[test_index], pred, np.unique(Y))
        resultado = relatorioDesempenho(cm, classes, False)

        resultados.append(resultado)
    
    return resultados, classes

start_time = datetime.now()
print('Naive Bayes em andamento...')
resultados, classes = multinomial_naiveBayes(folds, X, Y)
mediaFolds(resultados, classes)
end_time = datetime.now()
print('Duração - Naive Bayes: {}'.format(end_time - start_time))

Naive Bayes em andamento...

	Revocacao   Precisao   F-medida   Classe
	0.787       0.834      0.808      0
	0.840       0.801      0.818      1
	---------------------------------------------------------------------
	0.814       0.817      0.813      Média macro
	0.814       0.814      0.814      Média micro

	Acuracia: 0.814
Duração - Naive Bayes: 0:00:11.462167


In [13]:
# KNN
def K_Nearest_Neighbors(folds, X, Y):
    '''
        Dado os folds, X e Y.
        Então é executado o algoritmo KNN para cada fold.
        
        Entrada: folds - Folds para serem testados.
                 X - Matriz de conjunto de dados.
                 Y - Lista com as classes correspondente aos dados da matriz X.
        Saida: resultados - Lista de resultados para cada fold.
               classes - Lista com as classes baseada na lista Y.
    '''
    resultados = []
    classes = np.unique(Y)

    for train_index, test_index in folds:
        # Classificação
        pred = []
        K = 3
        for x in X[test_index]:
            y, ind_viz = knn(x, X, Y, K)
            pred.append(y)

        cm = get_confusionMatrix(Y[test_index], pred, np.unique(Y))
        resultado = relatorioDesempenho(cm, classes, False)

        resultados.append(resultado)
    
    return resultados, classes

print('KNN em andamento...')
start_time = datetime.now()
resultados, classes = K_Nearest_Neighbors(folds, X, Y)
end_time = datetime.now()
print('Duração - KNN: {}'.format(end_time - start_time))
mediaFolds(resultados, classes)

KNN em andamento...
Duração - KNN: 0:02:23.667749

	Revocacao   Precisao   F-medida   Classe
	0.766       0.863      0.811      0
	0.878       0.790      0.831      1
	---------------------------------------------------------------------
	0.822       0.826      0.821      Média macro
	0.822       0.822      0.822      Média micro

	Acuracia: 0.822


0.821875

In [14]:
'''
REDE NEURAL
'''

'\nREDE NEURAL\n'

In [15]:
# Rede Neural
def redeNeural(folds, X, Y, vLambda, tamanho_intermediaria):
    '''
        Dado os folds, X e Y, vLambda e tamanho_intermediaria.
        Em que:
            vLambda é coeficiente de regularização
            tamanho_intermediaria é o número de neurônios na camada intemediária
        Então é executado o algoritmo naive bayes para cada fold.
        
        Entrada: folds - Folds para serem testados.
                 X - Matriz de conjunto de dados.
                 Y - Lista com as classes correspondente aos dados da matriz X.
                 
        Saida: resultados - Lista de resultados para cada fold.
               classes - Lista com as classes baseada na lista Y.
    '''
    resultados = []
    
    classes = np.unique(Y)
    tamanho_entrada = X.shape[1]

    
    Theta1 = inicializaPesosAleatorios(tamanho_entrada, tamanho_intermediaria, randomSeed = 10);
    Theta2 = inicializaPesosAleatorios(tamanho_intermediaria, 2, randomSeed = 20);
    Thetas = np.concatenate([np.ravel(Theta1), np.ravel(Theta2)])
    
    start_time = datetime.now()
    for train_index, test_index in folds:
        
        # Otimização dos Thetas
        
        Theta1, Theta2 = rna_treino(Thetas, tamanho_entrada, tamanho_intermediaria, 2, X[train_index], Y[train_index], vLambda)
        save_raw(Theta1, 'RNAResults/' + 'v' + str(vLambda) + 'n' + str(tamanho_intermediaria) + 'Theta1')
        save_raw(Theta2, 'RNAResults/' + 'v' + str(vLambda) + 'n' + str(tamanho_intermediaria) + 'Theta2')
        
        # Classificação
        pred = rna_predicao(Theta1, Theta2, X[test_index])
        #display(pred)

        cm = get_confusionMatrix(Y[test_index], pred, np.unique(Y))
        resultado = relatorioDesempenho(cm, classes, False)

        resultados.append(resultado)
        end_time = datetime.now()
        print('Duração - Fold em RNA: {}'.format(end_time - start_time))
    
    return resultados, classes

#print('RNA em andamento...')
#start_time = datetime.now()
#resultados, classes = redeNeural(folds, X, Y, 1, 10)
#end_time = datetime.now()
#print('Duração - RNA: {}'.format(end_time - start_time))
#mediaFolds(resultados, classes)

# ----------------------------
# Rotina de testes
#----------------------------
vlambdas = [1, 3, 5 , 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 42]
inter_neuros = [2, 3, 5, 7 ,10, 15, 25]
#inter_neuros = [10]

#cria matriz para salvar no csv
save_matrix = [[0 for x in range(len(inter_neuros)+1)] for y in range(len(vlambdas)+1)]
save_matrix[0][0] = -1
for i in range(len(inter_neuros)):
    save_matrix[0][i+1] = inter_neuros[i]
for i in range(len(vlambdas)):
    save_matrix[i+1][0] = vlambdas[i]
display(save_matrix)

display('#############################')
display('Iniciando rotina de Testes...')
display('#############################')
start_all = datetime.now()
for v in range(len(vlambdas)):
    for i in range(len(inter_neuros)):
        display('------------------------------------------------------------')
        display('RNA em andamento com vLambda = ' + str(vlambdas[v]) + ' e ' + str(inter_neuros[i]) + ' neurônios na camada intermediária...' )
        start_time = datetime.now()
        resultados, classes = redeNeural(folds, X, Y, vlambdas[v], inter_neuros[i])
        end_time = datetime.now()
        print('Duraçãos Total - RNA: {}'.format(end_time - start_time))
        save_matrix[v+1][i+1] = mediaFolds(resultados, classes)
        display(save_matrix)
        
display('############################')
display('Rotina de testes finalizadas')
end_time = datetime.now()
print('Duração Total - RNA: {}'.format(end_time - start_all))
save_raw(save_matrix, 'RNAResults/Results')
display('############################')

[[-1, 2, 3, 5, 7, 10, 15, 25],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0, 0, 0],
 [5, 0, 0, 0, 0, 0, 0, 0],
 [7, 0, 0, 0, 0, 0, 0, 0],
 [9, 0, 0, 0, 0, 0, 0, 0],
 [11, 0, 0, 0, 0, 0, 0, 0],
 [13, 0, 0, 0, 0, 0, 0, 0],
 [15, 0, 0, 0, 0, 0, 0, 0],
 [17, 0, 0, 0, 0, 0, 0, 0],
 [19, 0, 0, 0, 0, 0, 0, 0],
 [21, 0, 0, 0, 0, 0, 0, 0],
 [23, 0, 0, 0, 0, 0, 0, 0],
 [25, 0, 0, 0, 0, 0, 0, 0],
 [27, 0, 0, 0, 0, 0, 0, 0],
 [29, 0, 0, 0, 0, 0, 0, 0],
 [31, 0, 0, 0, 0, 0, 0, 0],
 [33, 0, 0, 0, 0, 0, 0, 0],
 [42, 0, 0, 0, 0, 0, 0, 0]]

'#############################'

'Iniciando rotina de Testes...'

'#############################'

'------------------------------------------------------------'

...

[[-1, 2, 3, 5, 7, 10, 15, 25],
 [1,
  0.8493749999999999,
  0.849375,
  0.8475000000000001,
  0.8481250000000001,
  0.8481250000000001,
  0.8475000000000001,
  0.8487500000000001],
 [3, 0.850625, 0.85, 0.850625, 0.849375, 0.85, 0.8506250000000002, 0.85],
 [5,
  0.849375,
  0.85375,
  0.8525,
  0.8518749999999999,
  0.85125,
  0.853125,
  0.853125],
 [7,
  0.85375,
  0.8537500000000001,
  0.8549999999999999,
  0.853125,
  0.85375,
  0.8543750000000001,
  0.85625],
 [9,
  0.85375,
  0.85625,
  0.8550000000000001,
  0.85625,
  0.8556249999999999,
  0.8574999999999999,
  0.85625],
 [11,
  0.85625,
  0.8568749999999999,
  0.8556250000000001,
  0.8556250000000001,
  0.8556250000000001,
  0.8556250000000001,
  0.85625],
 [13,
  0.85625,
  0.8543749999999999,
  0.8556250000000001,
  0.85625,
  0.8581249999999999,
  0.8568749999999999,
  0.85625],
 [15,
  0.8574999999999999,
  0.85625,
  0.858125,
  0.858125,
  0.858125,
  0.858125,
  0.858125],
 [17, 0.8574999999999999, 0.85875, 0.859375, 0.85

'############################'

'Rotina de testes finalizadas'

Duração Total - RNA: 1:49:51.120683


'############################'

In [16]:
'''
REGRESSAO LOGISTICA
'''

'\nREGRESSAO LOGISTICA\n'

In [17]:
# REGRESSAO LOGISTICA
def regressaoLogistica(folds, X, Y):
    '''
        Dado os folds, X e Y.
        Então é executado o algoritmo para cada fold.
        
        Entrada: folds - Folds para serem testados.
                 X - Matriz de conjunto de dados.
                 Y - Lista com as classes correspondente aos dados da matriz X.
        Saida: resultados - Lista de resultados para cada fold.
               classes - Lista com as classes baseada na lista Y.
    '''
    
    resultados = []
    classes = np.unique(Y)
    m, n = X.shape
    X = np.column_stack( (np.ones(m),X) ) 
    MaxIter = 100 

    theta = np.zeros(n+1)
    result = scipy.optimize.minimize(fun=funcaoCusto, x0=theta, args=(X, Y),  
                method='BFGS', jac=True, options={'maxiter': MaxIter, 'disp':True})
    theta = result.x

    for train_index, test_index in folds:
        pred = []
        pred = predicao(theta, X[test_index])

        cm = get_confusionMatrix(Y[test_index], pred, np.unique(Y))
        resultado = relatorioDesempenho(cm, classes, False)

        resultados.append(resultado)
    
    return resultados, classes

print('Regressao Logistica em andamento...')
start_time = datetime.now()
resultados, classes = regressaoLogistica(folds, X, Y)
end_time = datetime.now()
print('Duração - Regressao Logistica: {}'.format(end_time - start_time))
mediaFolds(resultados, classes)

Regressao Logistica em andamento...
Optimization terminated successfully.
         Current function value: 0.000005
         Iterations: 61
         Function evaluations: 62
         Gradient evaluations: 62
Duração - Regressao Logistica: 0:01:11.762516

	Revocacao   Precisao   F-medida   Classe
	1.000       1.000      1.000      0
	1.000       1.000      1.000      1
	---------------------------------------------------------------------
	1.000       1.000      1.000      Média macro
	1.000       1.000      1.000      Média micro

	Acuracia: 1.000


1.0